# Webcam (on PYNQ-Z2)
```from composable_pipeline.video import Webcam```

In [1]:
import cv2
print(cv2.getBuildInformation())


General configuration for OpenCV 3.4.3 =====================================
  Version control:               unknown

  Extra modules:
    Location (extra):            /root/opencv_contrib-3.4.3/modules
    Version control (extra):     unknown

  Platform:
    Timestamp:                   2020-05-18T20:51:00Z
    Host:                        Linux 4.4.0-154-generic armv7l
    CMake:                       3.10.2
    CMake generator:             Unix Makefiles
    CMake build tool:            /usr/bin/make
    Configuration:               RELEASE

  CPU/HW features:
    Baseline:                    NEON
      requested:                 DETECT
      required:                  NEON
      disabled:                  VFPV3

  C/C++:
    Built as dynamic libs?:      YES
    C++11:                       YES
    C++ Compiler:                /usr/bin/c++  (ver 7.3.0)
    C++ flags (Release):         -fsigned-char -W -Wall -Werror=return-type -Werror=non-virtual-dtor -Werror=address -Werror=sequ

## Webcam._configure()

In [2]:
dev = 0
videoIn = cv2.VideoCapture(dev)

In [3]:
print(videoIn)

<VideoCapture 0xa8e623f0>


In [4]:
from pynq.lib.video import VideoMode
mode = VideoMode(1280,720,24,30); print(mode)

VideoMode: width=1280 height=720 bpp=24


In [5]:
if videoIn:
    videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, mode.width);
    videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, mode.height);
    videoIn.set(cv2.CAP_PROP_FPS, mode.fps)
print(videoIn.isOpened())

False


## Prepare `cifar2png`

In [6]:
import os
if not os.path.isdir('cifar10'):
    if not os.path.isdir('cifar2png'):
        ! git clone https://github.com/knjcode/cifar2png.git && git -C cifar2png checkout tags/0.0.4
    ! python3 cifar2png/cifar2png cifar10 cifar10 --name-with-batch-index
if not os.path.isfile('cifar10/test/test_batch_index_0000.png'):
    ! mv cifar10/test/*/* cifar10/test
    ! rmdir cifar10/test/*/

## CIFAR-10 as VideoCapture

In [7]:
if not videoIn.isOpened():
    videoIn.release()
    videoIn = cv2.VideoCapture('cifar10/test/test_batch_index_%04d.png', cv2.CAP_IMAGES)
print(videoIn.isOpened())

True


## Webcam.tie()

In [8]:
import threading
from _thread import start_new_thread
thread = threading.Lock(); print(thread)

<unlocked _thread.lock object at 0xa8eb3c20>


In [9]:
from time import sleep
ret = None; frame = None
def _tie():
    global ret, frame
    while running:
        ret, frame = videoIn.read()
        if not ret:
            videoIn.set(cv2.CV_CAP_PROP_POS_AVI_RATIO, 0)
        sleep(0.01)
    thread.release()

In [10]:
if not videoIn or not videoIn.isOpened():
    raise SystemError('The Webcam is not started')
elif not thread.locked():
    thread.acquire()
    running = True
    try:
        start_new_thread(_tie, ())
    except:
        import traceback
        print (traceback.format_exc())

## Display the Webcam

In [12]:
from IPython.display import display, clear_output
from matplotlib import pyplot
%matplotlib inline
for _ in range(10):
    if frame.any():
        pyplot.imshow(frame[:,:,::-1])
        pyplot.show()
        display(pyplot.gcf())
        clear_output(wait=True)
    sleep(0.1)

Unhandled exception in thread started by <function _tie at 0xa85a4ae0>


## Webcam.stop()

In [ ]:
if videoIn:
    running = False
    while thread.locked():
        sleep(0.05)
    videoIn.release()
    videoIn = None

In [ ]:
print(videoIn)